In [14]:
import os

from dotenv import load_dotenv

load_dotenv()

ID = os.environ.get("ID")
PW = os.environ.get("PW")

테스트 한다고 계속 돌리다가 clipboard를 이용한 우회 방법 또한 CAPTCHA가 발생할 수 있는데  
이는 여러 번 로그인을 시도한 케이스라 `실제로 로그인을 한 번 진행` 후 하면 잘 우회된다.

In [17]:
import time

import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


NAVER_LOGIN_URL = "https://nid.naver.com/nidlogin.login?mode=form&url=https://www.naver.com/"

driver =webdriver.Chrome()
driver.get(NAVER_LOGIN_URL)

# id 입력
id_element = driver.find_element(By.CSS_SELECTOR, "#id")
pyperclip.copy(ID)
id_element.send_keys(Keys.COMMAND, 'v')
time.sleep(1)

# pw 입력
pw_element = driver.find_element(By.CSS_SELECTOR, "#pw")
pyperclip.copy(PW)
pw_element.send_keys(Keys.COMMAND, 'v')
time.sleep(1)

# trigger login submit
pw_element.send_keys(Keys.ENTER)



In [16]:
driver.quit()

# pip install pypeclip